# Flux1.x devのサンプル利用コード
##利用方法
・基本的には全てのセルを実行することで利用できます。

・GPUを利用しましょう。（利用制限があるのでこまめにインスタンスは削除してください）

（変更する際は，「ランタイム」→「ランタイムのタイプを変更」からGPUを選択してください）

##注意点
・本ファイルの相対的な場所は変更しないでください。

・本ファイルの名前を変更しないでください。

変更する場合は，[2]セル目のファイル名も一緒に変更してください。

・[1]セル目の実行時にセッションの再起動を求められることがあります。

その場合はポップアップに従って再起動したあと，全てのセルを再度実行してください。

In [1]:
#Google Driveのフォルダをマウント（認証入る）
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
#SDXL で必要なモジュールのインストール
%rm -r /content/diffusers
%cd /content/
!git clone https://github.com/huggingface/diffusers.git
!pip install -U bitsandbytes optimum-quanto peft tensorflow-metadata transformers scikit-learn ftfy accelerate invisible_watermark safetensors controlnet-aux mediapipe timm



rm: cannot remove '/content/diffusers': No such file or directory
/content
Cloning into 'diffusers'...
remote: Enumerating objects: 70235, done.
remote: Counting objects: 100% (9551/9551), done.
remote: Compressing objects: 100% (1168/1168), done.
remote: Total 70235 (delta 9020), reused 8565 (delta 8302), pack-reused 60684 (from 1)
Receiving objects: 100% (70235/70235), 49.05 MiB | 16.35 MiB/s, done.
Resolving deltas: 100% (52050/52050), done.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.4/42.4 kB 3.5 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of mediapipe to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 MB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.7/109.7 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.4/296.4 kB 21.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 102.9 MB/s eta 0:00:00
   

下記のコードを実行すると，認証が求められるので，認証してください。（Google colabにマイドライブの中を読めるようにする認証です）

In [3]:
from huggingface_hub import login
from google.colab import userdata
HF_LOGIN = userdata.get('HF_LOGIN')
login(HF_LOGIN)

# カレントディレクトリを本ファイルが存在するディレクトリに変更する。
import glob
import os
pwd = os.path.dirname(glob.glob('/content/drive/MyDrive/colabzenn/colab_fluxdev_sample2/FluxDev_sample2.ipynb', recursive=True)[0])
print(pwd)

%cd $pwd
!pwd

import sys
sys.path.append("/content/diffusers/src")

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful
/content/drive/MyDrive/colabzenn/colab_fluxdev2_sample
/content/drive/MyDrive/colabzenn/colab_fluxdev2_sample
/content/drive/MyDrive/colabzenn/colab_fluxdev2_sample


In [4]:
#モジュールをimportする
from module.module_flux import FLUX
import time

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

モデルを変更する場合は，下記のconfigを変更してください。

「;」を付けるとコメントアウトできます。


In [5]:
#モデルの設定を行う。

config_text = """
[FLUX]
device = auto
n_steps=24
seed=42

;model_mode = t2i
model_mode = i2i

strength = 0.6

;from_single_file = False
;base_model_path = black-forest-labs/FLUX.1-schnell
base_model_path = black-forest-labs/FLUX.1-dev

use_controlnet = False
controlnet_path = Shakker-Labs/FLUX.1-dev-ControlNet-Union-Pro

;control_mode0 = canny
;control_mode1 = tile
control_mode2 = depth
;control_mode3 = blur
;control_mode4 = pose
;control_mode5 = gray
;control_mode6 = lq

;lora_weight_repo0 = alfredplpl/flux.1-dev-modern-anime-lora
;lora_weight_path0 = modern-anime-lora-2.safetensors
;lora_weight_repo1 = aleksa-codes/flux-ghibsky-illustration
;lora_weight_path1 = lora.safetensors
;lora_scale0 = 1.0
;lora_scale1 = 1.0
;trigger_word0 = "modern anime style"
;trigger_word1 = "GHIBSKY style painting"
;lora_weight_repo2 = None
;lora_weight_path2 = ./inputs/QTCanimation_lora_v1-PAseer.safetensors
;lora_scale2 = 1.0
;trigger_word2 = ""

;select_solver = LCM
;select_solver = DPM
;select_solver = Eulera
select_solver = FMEuler

use_karras_sigmas = True
scheduler_algorithm_type = dpmsolver++
solver_order = 2

cfg_scale = 3.5
;cfg_scale = 1.0
width = 832
height = 1216
output_type = pil
aesthetic_score = 6
negative_aesthetic_score = 2.5

save_latent_simple = False
save_latent_overstep = False


"""

with open("configs/config.ini", "w", encoding="utf-8") as f:
  f.write(config_text)

生成する画像を決定するプロンプトを指定します。

negative_promptを設定することで，ここに書かれている内容を回避できる。


In [6]:
#読み上げるプロンプトを設定する。

# not use controlnet
#main_prompt = """
#modern anime style, a cute Japanese idle singer girl, long Yellowish-white hair with red small ribbon,red eyes,small red hat ,She is singing ,holding a microphone, wearing white idle costume, standing on the stage ,This is a flashy concert with 3D holograms and laser effects
#"""

#use controlnet depth
#main_prompt = """
#modern anime style, a cute Japanese idle singer girl, long Yellowish-white hair with red small ribbon,red eyes,small red hat ,She sings with a smile on her face , wearing white idle costume, standing on the stage ,This is a flashy concert with 3D holograms and laser effects
#"""

#use i2i + lora
main_prompt = """
modern anime style, a cute Japanese idle singer girl, long Yellowish-white hair with red small ribbon,red eyes,small red hat ,She sings with a smile on her face ,make a peace sign with her left hand, wearing white idle costume, standing on the stage ,This is a flashy concert with 3D holograms and laser effects
"""



#double lora and anime lora only
#main_prompt = """
#a cute Japanese idle singer girl, long Yellowish-white hair with red small ribbon,red eyes,small red hat ,She sings with a smile on her face , wearing white idle costume, standing on the stage
#"""


#use controlnet canny
#main_prompt = """
#modern anime style, a cute Japanese idle singer girl, long Yellowish-white hair with red small ribbon,red eyes
#"""

#コントロールネットに入力する参照画像は0から6までの数字を想定（0.png,6.webpなど。数字はcontrolnetのモードと一致させてください）
#参照）コントロールネットのモード　canny (0), tile (1), depth (2), blur (3), pose (4), gray (5), low quality (6).
input_refer_image_folder = "./inputs/controlnet/refer"
#コントロールネットに入力する変換後の参照画像の格納フォルダ。命名規則は上記と同様
output_refer_image_folder = "./inputs/controlnet/refer_prepared"

#image to imageモードを利用する際の画像path
i2i_image_path = "./inputs/imagetoimage/inputs1.png"





In [7]:
flux = FLUX()
#指定しているControlNetで利用できる"./inputs/refer_prepared/0.png"などがすでにある場合は、下記はコメントアウトしても良い
#flux.prepare_multi_referimage(input_refer_image_folder = input_refer_image_folder,output_refer_image_folder = output_refer_image_folder, low_threshold = 100, high_threshold = 200, noise_level=25, blur_radius=5)



i2i mode


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


transformer/config.json:   0%|          | 0.00/378 [00:00<?, ?B/s]

(…)ion_pytorch_model.safetensors.index.json:   0%|          | 0.00/121k [00:00<?, ?B/s]

Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

...

tokenizer_2/tokenizer_config.json:   0%|          | 0.00/20.8k [00:00<?, ?B/s]

vae/config.json:   0%|          | 0.00/820 [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

You set `add_prefix_space`. The tokenizer needs to be converted from the slow tokenizers


loaded base model
cpu offloading


In [12]:
controlnet_conditioning_scale = [0.5]
temp_strength = 0.6
#temp_strength = None

for i in range(5):
      start = time.time()
      image = flux.generate_image(main_prompt,c_image_path = output_refer_image_folder, i_image_path = i2i_image_path, controlnet_conditioning_scale = controlnet_conditioning_scale, temp_strength = temp_strength)
      print("generate image time: ", time.time()-start)
      image.save("./outputs/FLUX_result_{}.png".format(i))

use i2i mode
Image loaded from file path.


  0%|          | 0/15 [00:00<?, ?it/s]

generate image time:  54.74718999862671
use i2i mode
Image loaded from file path.


  0%|          | 0/15 [00:00<?, ?it/s]

generate image time:  54.64600729942322
use i2i mode
Image loaded from file path.


...

generate image time:  54.82111692428589
use i2i mode
Image loaded from file path.


  0%|          | 0/15 [00:00<?, ?it/s]

generate image time:  54.829524517059326
use i2i mode
Image loaded from file path.


  0%|          | 0/15 [00:00<?, ?it/s]

generate image time:  54.650004148483276


In [9]:
from google.colab import runtime
get = 0
if get == 1:
    runtime.unassign()